# **Tutorial 3 : A Zero-Dimensional Energy Balance Model**


**Week 1, Day 5, Climate Modeling**

**Content creators:** Jenna Pearson

**Content reviewers:** Draco Xu, Will Gregory, Ohad Zivan, Chi Zhang

**Content editors:** Brodie Pearson, and Abigail Bodner, Ohad Zivan, Chi Zhang

**Production editors:** TBD

**Our 2023 Sponsors:** TBD

#**Tutorial Objectives**

In this tutorial students will learn about the heat capacity of the atmosphere and oceans, how this related to temperature changes over time, and set up their first climate model.

By the end of this tutorial students will be able to:
* Calculate the heat capacity of the ocean and atmospehre.
* Create and run a time-dependent model of the change in global mean surfact temperture.
* Describe the influence of transmissivity and albedo on the equilibrium temperature from this model.

# Setup

## Package Imports

In [ ]:
import xarray as xr                     # used to manipulate data and open datasets
import numpy as np                      # used for algeba and array operations
import matplotlib.pyplot as plt         # used for plotting

## Figure Settings

In [ ]:
import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle")

# Section 1 : Building the Model

## Section 1.1 : Moving Forward in Time

One of the crucial things missing from the simple model we have looked at so far is it's ability to change with time. As the composition of the atmosphere (among other things) changes with time, in response, so does the energy balance and global mean surface temperature. This is shown by the thick black lines in the figure below, where the time series of observed global mean surface air temperature change from the 1850-1900 reference period is plotted. Figures like this are called 'hockey stick' figures due to their shape: a relatively stable period followed by a steep increase. In order to incorporate this into our model, we need to include a mathetmatical representation of how the change in energy relates to a change in temperature over time.

![Observerd and simulated change in temparture](https://www.ipcc.ch/report/ar6/wg1/downloads/figures/IPCC_AR6_WGI_Figure_3_4.png)
Figure 3.4 | Observed and simulated time series of the anomalies in annual and global mean surface air temperature (GSAT). All anomalies are differences from the 1850–1900 time-mean of each individual time series. The reference period 1850–1900 is indicated by grey shading. (a) Single simulations from CMIP6 models (thin lines) and the multi-model mean (thick red line). Observational data (thick black lines) are from the Met Office Hadley Centre/Climatic Research Unit dataset (HadCRUT5), and are blended surface temperature (2 m air temperature over land and sea surface temperature over the ocean). All models have been subsampled using the HadCRUT5 observational data mask. Vertical lines indicate large historical volcanic eruptions. CMIP6 models which are marked with an asterisk are either tuned to reproduce observed warming directly, or indirectly by tuning equilibrium climate sensitivity. Inset: GSAT for each model over the reference period, not masked to any observations. (b) Multi-model means of CMIP5 (blue line) and CMIP6 (red line) ensembles and associated 5th to 95th percentile ranges (shaded regions). Observational data are HadCRUT5, Berkeley Earth, National Oceanic and Atmospheric Administration NOAAGlobalTemp-Interim and Kadow et al. (2020). Masking was done as in (a). CMIP6 historical simulations were extended with SSP2-4.5 simulations for the period 2015–2020 and CMIP5 simulations were extended with RCP4.5 simulations for the period 2006–2020. All available ensemble members were used (see Section 3.2). The multi-model means and percentiles were calculated solely from simulations available for the whole time span (1850–2020). Figure is updated from Bock et al. (2020), their Figures 1 and 2. CC BY 4.0https://creativecommons.org/licenses/by/4.0/. Further details on data sources and processing are available in the chapter data table (Table 3.SM.1).




We can represent the change in temperature over time as the net heat input or loss from radiation ($ASR-OLR$) multiplied by the heat capacity of the Earth system.

The heat capacity of a medium is it's ability to increase in temperture given an input of heat. Not all components of the Earth system (for example land, ocean, atmosphere) have the same heat capacity.

Mathematically the rate of change of global mean surface temperature ($T$) over time is given as

\begin{align}
\text{rate of change of }T = \frac{1}{\text{heat capacity}}\cdot (\text{energy flux in - energy flux out}) = \frac{1}{C} \cdot ASR - OLR
\end{align}

where $C$ is the heat capacity of the Earth system. Note here that when $ASR > OLR$, then the system is gaining heat. Conversly when $ASR < OLR$, then the system is losing heat over time.

To calculate the heat capacity for the Earth system, we will assume the it is a combination of atmophere and ocean only, that is $C = C_{oc} + C_{atmo}$. 

Generally, the heat capacity of a medium is the specific heat of that medium times the total mass.

For the atmosphere we have $C_{atm} = c_{atm} \cdot \frac{W_{atm}}{g} $ where $c_{atm}$ is the specific heat of the atmosphere, $W_{atm}$ is the weight of a column of air, and $g$ is the acceleration due to gravity.

For the ocean we have $C_{oc} = c_{oc} \cdot \rho_{oc} \cdot d_{oc}$ where $c_{oc}$ is the specific heat of the ocean, $\rho_{oc}$ is the density of seawater, and $d_{oc}$ is a representive depth of the ocean. 

We will use this definition to find the heat capactities of the atmosphere and ocean.

In [15]:
# heat capacity of the ocean
c_oc = 3850                       #  specific heat of seawater in J/kg/K
rho_oc = 1025                     #  average density of seawater in kg/m3
d_oc = 70                         #  depth of water in m (here representative of the mized layer)
C_oc = c_oc * rho_oc * d_oc       #  heat capacity of the ocean

# heat capacity of the atmosphere
c_atm = 1004                       #  specific heat of the atmosphere at constant pressure in J/kg/K
W_atm = 100000                     #  weight (pressure) of atmospheric column in Pa
g = 9.81                           #  acceleration due to gravity in m/s^2
C_atm = c_atm * (W_atm / g)        #  heat capacity of the atmosphere

# total heat capacity
C = C_oc + C_atm

#print results.

print('Ocean Heat Capactiy:      ', C_oc,' J m^-2K^-2')
print('Atmosphere Heat Capacity: ', C_atm,'   J m^-2K^-2')
print('Total Heat Capactiy:      ', C,' J m^-2K^-2')

Ocean Heat Capactiy:       276237500  J m^-2K^-2
Atmosphere Heat Capacity:  10234454.638124362    J m^-2K^-2
Total Heat Capactiy:       286471954.63812435  J m^-2K^-2


### Coding Exercises 1.1

1.  Calculate the depth of the ocean needed for the ocean to have the same heat capacity at the atmosphere.

In [ ]:
#################################################
# Students: Fill in missing code (...) and comment or remove the next line
# Fill out function and remove
raise NotImplementedError("Student exercise: Calculate the depth of the ocean needed for the ocean to have the same heat capacity at the atmosphere.")
#################################################
  
# heat capacity of the atmosphere
c_atm = 1004                       #  specific heat of the atmosphere at constant pressure in J/kg/K
W_atm = 100000                     #  weight (pressure) of atmospheric column in Pa
g = 9.81                           #  height of atmosphere in m (representative of )
C_atm = c_atm * (W_atm / g)        #  heat capacity of the atmosphere

# find the depth of the ocean for equivalent atmospheric heat capacity
c_oc = 3850                       #  specific heat of seawater in J/kg/K
rho_oc = 1025                     #  average density of seawater in kg/m3

d_oc  = ...
d_oc 

In [16]:
# to_remove solution

# heat capacity of the atmosphere
c_atm = 1004                       #  specific heat of the atmosphere at constant pressure in J/kg/K
W_atm = 100000                     #  weight (pressure) of atmospheric column in Pa
g = 9.81                           #  height of atmosphere in m (representative of )
C_atm = c_atm * (W_atm / g)        #  heat capacity of the atmosphere

# find the depth of the ocean for equivalent atmospheric heat capacity
c_oc = 3850                       #  specific heat of seawater in J/kg/K
rho_oc = 1025                     #  average density of seawater in kg/m3

d_oc  = C_atm/(c_oc * rho_oc)      #  heat capacity of the ocean
d_oc 

2.5934633229330024

### Questions 1.1

1. In your own words, describe what the answer to your coding exercise means.

In [ ]:
# to_remove explanation

"""
1. We could heat the entire atmosphere by 1 degree with the energy required to raise the upper 2 and a half meters of the ocean by 1 degree. 
   Relative to the atmosphere, the ocean can take in an enormous amount of energy before raising it's temperature appreciably.
"""

## Section 1.2 : The Numerical Model

Knowing the heat capacity, and the descriptions of $OLR$ and $ASR$ from previous tutorials, we can write the equation

\begin{align}
\text{rate of change }T = \frac{\text{change in }T}{\text{change in time}}=\frac{dT}{dt}= \frac{1}{C}(ASR - OLR)
\end{align}

Numerically, we can use this equation to compute the global mean surface temprature after a small interval of time by adding on the amount of energy gained or lost multiplied by the time interval itself. 

The particular method of numerically defining the time and temperature intervals (changes) is called **discretization**, and the way we have chosen to do this is called the **Euler method**.The exact details of this method are beyond the scope of this tutorial, and we will use the method without further elaboration.

The Euler method assumes we can use $\text{change in }T = T_{n+1} - T_{n}$ and $\text{change in t} = t_{n+1} - t_{n}$ where $t$ is time. Thus, if we know the time interval and the current temperature ($T_n$), we can predict the temperature at the end of our time interval, ($T_{n+1}$).

In [17]:
# define albedo
alpha = 0.2941 # unitless number between 0 and 1 (calculated previously from observations in tutorial 2)

# define transmissivity (calculated previously from observations in tutorial 1)
tau = 0.6127 # unitless number between 0 and 1

# create a function to find the new tempeature based on the previous using Euler's method.
def step_forward(T,alpha,tau):

    # define the observed insolation based on observations from the IPCC AR6 Figure 7.2
    Q = 340 # W m^-2

    # define the Stefan-Boltzmann Constant, noting we are using 'e' for scientific notation
    sigma = 5.67e-8 # W m^-2 K^-4

    ASR = (1-alpha)*Q
    OLR = tau * sigma * T**4

    T_new = T + dt / C * ( ASR - OLR )

    return T_new

We can now use a loop to apply this function many times over by specifying an initial temperature and a time interval. 

In [ ]:
# define the time interval, one year expressed in seconds
dt = 60. * 60. * 24. * 365.

# define the number of timesteps (years) to run the model
numtsteps = 15

# for converting number of seconds in a year
sec_2_yr = 3.154e7

# define empty arrays to store the time series of temperature and the corresponding years
T_series = np.zeros((numtsteps+1))
t_series = np.zeros((numtsteps+1))

# define the intial temperature
T_series[0] = 288.

# run the model
for n in range(numtsteps):
    t_series[n+1] = (n+1)*dt/sec_2_yr
    T_series[n+1] = step_forward( T_series[n], alpha = alpha, tau = tau )

# display the temeprature time series
print(T_series)

In [ ]:
print(t_series)

In [ ]:
# plot the results
plt.plot(t_series, T_series)
plt.xlabel('Years')
plt.ylabel('Global mean temperature (K)');

## Questions

1. What impact do you think the time interval ($dt$) has on the accuracy of the model results?
2. Do you think a smaller or larger time interval than one year would provide more accurate results?

## Coding Exercises

1.  Using a for loop, run the model for 200 years with two different three intervals ($dt$) of a half year, 1 year and 5 years and plot the results.

In [ ]:
# # one year expressed in seconds
# one_yr = 60. * 60. * 24. * 365.

# # legend labels
# labels = ['dt = half-year','dt = one year','dt = five years']

# # define the number of timesteps (years) to run the model
# numtsteps = np.array([10,5,1])*3

#  # for converting number of seconds in a year
# sec_2_yr = 3.154e7

# # loop through each choice of time step
# for dd,dt ...

#   # define empty arrays to store the time series of temperature and the corresponding years
#   T_series = np.zeros((numtsteps[dd]+1))
#   t_series = np.zeros((numtsteps[dd]+1))

#   # define the intial temperature
#   T_series[0] = 288.

#   # run the model
#   for n in range(numtsteps[dd]):
#       t_series[n+1] = (n+1)*dt/sec_2_yr
#       T_series[n+1] = step_forward( T_series[n], alpha = alpha, tau = tau )

#   plt.plot(t_series, T_series, label = labels[dd])

# plt.xlabel('Years')
# plt.ylabel('Global mean temperature (K)');
# plt.legend()

In [ ]:
# to_remove solution

# one year expressed in seconds
one_yr = 60. * 60. * 24. * 365.

# legend labels
labels = ['dt = half-year','dt = one year','dt = five years']

# define the number of timesteps (years) to run the model
numtsteps = np.array([10,5,1])*3

 # for converting number of seconds in a year
sec_2_yr = 3.154e7

# loop through each choice of time step
for dd,dt in enumerate([one_yr*0.5,one_yr,one_yr*5]):

  # define empty arrays to store the time series of temperature and the corresponding years
  T_series = np.zeros((numtsteps[dd]+1))
  t_series = np.zeros((numtsteps[dd]+1))

  # define the intial temperature
  T_series[0] = 288.

  # run the model
  for n in range(numtsteps[dd]):
      t_series[n+1] = (n+1)*dt/sec_2_yr
      T_series[n+1] = step_forward( T_series[n], alpha = alpha, tau = tau )

  plt.plot(t_series, T_series, label = labels[dd])

plt.xlabel('Years')
plt.ylabel('Global mean temperature (K)');
plt.legend()

# Section 3 : Revisiting the Climate Change Scenario from Tutorial 2

In tutorial 2 we looked at how changing the transmissivity ($\tau$) affected the equilibrium temperature. Now we can use our time-dependent model to investigate this more fully. Reuse the model, this time setting $\tau=0.57$

In [ ]:
# define albedo
alpha = 0.2941 # unitless number between 0 and 1 (calculated previously from observations in tutorial 2)

# define transmissivity (calculated previously from observations)
tau = 0.57 # unitless number between 0 and 1

# define the time interval, one year expressed in seconds
dt = 60. * 60. * 24. * 365.

# define the number of timesteps (years) to run the model
numtsteps = 15

# for converting number of seconds in a year
sec_2_yr = 3.154e7

# define empty arrays to store the time series of temperature and the corresponding years
T_series = np.zeros((numtsteps+1))
t_series = np.zeros((numtsteps+1))

# define the intial temperature
T_series[0] = 288.

# run the model
for n in range(numtsteps):
    t_series[n+1] = (n+1)*dt/sec_2_yr
    T_series[n+1] = step_forward( T_series[n], alpha = alpha, tau = tau )

plt.plot(t_series, T_series)

plt.xlabel('Years')
plt.ylabel('Global mean temperature (K)');

## Questions

1.  How does the long-term temperature here compare to the value you found from tutorial 2?


## Coding Exercises

1. Run the model again keeping the original $\tau=0.6114$ but now change $\alpha=0.2$.

In [ ]:
#insert code

In [ ]:
#to_remove solution

# define albedo
alpha = 0.2 # unitless number

# define transmissivity (calculated previously from observations in tutorial 1)
tau = 0.6127 # unitless number between 0 and 1

# define the time interval, one year expressed in seconds
dt = 60. * 60. * 24. * 365.

# define the number of timesteps (years) to run the model
numtsteps = 15

# for converting number of seconds in a year
sec_2_yr = 3.154e7

# define empty arrays to store the time series of temperature and the corresponding years
T_series = np.zeros((numtsteps+1))
t_series = np.zeros((numtsteps+1))

# define the intial temperature
T_series[0] = 288.

# run the model
for n in range(numtsteps):
    t_series[n+1] = (n+1)*dt/sec_2_yr
    T_series[n+1] = step_forward( T_series[n], alpha = alpha, tau = tau )

plt.plot(t_series, T_series)

plt.xlabel('Years')
plt.ylabel('Global mean temperature (K)');